In [19]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import psycopg2 as pg2
import warnings

In [20]:
def extrair_dados():
#Lendo arquivos CSV
    hits=pd.read_csv('C:/Users/andre/OneDrive/Área de Trabalho/DP6/hits.csv')
    products=pd.read_csv('C:/Users/andre/OneDrive/Área de Trabalho/DP6/products.csv')
    sessions=pd.read_csv('C:/Users/andre/OneDrive/Área de Trabalho/DP6/sessions.csv')
    #Remodelar tabelas, trazer somente informações que serão usadas
    products=products[['fullVisitorId','visitId','hitNumber','productPrice','productQuantity','productRevenue','v2ProductCategory','v2ProductName']]
    sessions=sessions[['fullVisitorId','visitId','date','visitNumber','channelGrouping','geoNetwork_country','geoNetwork_continent']]
    hits=hits[['hitNumber','fullVisitorId','visitId','transaction_transactionRevenue']]
    hits.rename(columns={'transaction_transactionRevenue':'Revenue'},inplace=True)
    #Considerar somente vendas que se concretizaram
    products=products[(products['productQuantity']>0) & (products['productRevenue']>0)]
    hits=hits[hits['Revenue']>0]
    #Alterando tipo dos dados
    hits['Revenue']=hits['Revenue'].astype(np.float64)
    hits[['hitNumber','fullVisitorId','visitId']]=hits[['hitNumber','fullVisitorId','visitId']].astype(str)
    products[['fullVisitorId','visitId','hitNumber']]=products[['fullVisitorId','visitId','hitNumber']].astype(str)
    products[['productPrice','productQuantity','productRevenue']]=products[['productPrice','productQuantity','productRevenue']].astype(np.float64)
    sessions[['fullVisitorId','visitId','visitNumber']]=sessions[['fullVisitorId','visitId','visitNumber']].astype(str)
    sessions[['date']]=sessions[['date']].astype(np.int64)
    return hits, products, sessions

In [21]:
def postgre_products():
        #Conectando com o PostgreSQL
        conn = pg2.connect(database='DP6', user='postgres', password='200613')
        cur=conn.cursor()
        #Query no PostgreSQL
        cur.execute('''
                CREATE TABLE IF NOT EXISTS products (
                fullVisitorId VARCHAR,
                visitId VARCHAR,
                hitNumber VARCHAR,
                productPrice FLOAT,
                productQuantity FLOAT,
                productRevenue FLOAT,
                v2ProductCategory VARCHAR,
                v2ProductName VARCHAR)
                ''')
        #Fechando conexão
        cur.close()
        conn.commit()

In [22]:
def postgre_hits():
        #Conectando com o PostgreSQL
        conn = pg2.connect(database='DP6', user='postgres', password='200613')
        cur=conn.cursor()
        #Query no PostgreSQL
        cur.execute('''
                CREATE TABLE IF NOT EXISTS hits (
                hitNumber VARCHAR,
                fullVisitorId VARCHAR,
                visitId VARCHAR,
                Revenue FLOAT)
                ''')
        #Fechando conexão
        cur.close()
        conn.commit()

In [23]:
def postgre_sessions():
        #Conectando com o PostgreSQL
        conn = pg2.connect(database='DP6', user='postgres', password='200613')
        cur=conn.cursor()
        #Query no PostgreSQL
        cur.execute('''
                CREATE TABLE IF NOT EXISTS sessions (
                fullVisitorId VARCHAR,
                visitId VARCHAR,
                date FLOAT,
                visitNumber VARCHAR,
                channelGrouping VARCHAR,
                geoNetwork_country VARCHAR,
                geoNetwork_continent VARCHAR) 
                ''')
        #Fechando conexão
        cur.close()
        conn.commit()

In [24]:
def inserir_hits(hits):
    #Conectando com o PostgreSQL
    conn = pg2.connect(database="DP6", user="postgres", password="200613")
    cur=conn.cursor()
    #Query no PostgreSQL
    insert_query = f"""insert into hits(
        hitNumber,
        fullVisitorId,
        visitId,
        Revenue) values (%s,%s,%s,%s)"""
    cur.executemany(insert_query, hits.values)
    #Fechando conexão
    cur.close()
    conn.commit() 

In [25]:
def inserir_products(products):
    #Conectando com o PostgreSQL
    conn = pg2.connect(database="DP6", user="postgres", password="200613")
    cur=conn.cursor()
    #Query no PostgreSQL
    insert_query = f"""insert into products(
        fullVisitorId,
        visitId,
        hitNumber,
        productPrice,
        productQuantity,
        productRevenue,
        v2ProductCategory,
        v2ProductName) values (%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur.executemany(insert_query, products.values)
    #Fechando conexão
    cur.close()
    conn.commit() 

In [26]:
def inserir_sessions(sessions):
    #Conectando com o PostgreSQL
    conn = pg2.connect(database="DP6", user="postgres", password="200613")
    cur=conn.cursor()
    #Query no PostgreSQL
    insert_query = f"""insert into sessions(
        fullVisitorId,
        visitId,
        date,
        visitNumber,
        channelGrouping,
        geoNetwork_country,
        geoNetwork_continent) values (%s,%s,%s,%s,%s,%s,%s)"""
    cur.executemany(insert_query, sessions.values)
    #Fechando conexão
    cur.close()
    conn.commit() 

In [27]:
#Ignorando avisos
warnings.filterwarnings("ignore")
#Retirando dados dos arquivos CSV
hits, products, sessions = extrair_dados()
#Criar base em Pg4Admin
postgre_hits()
postgre_products()
postgre_sessions()
#Inserir dados em base PostgreSQL
inserir_hits(hits)
inserir_products(products)
inserir_sessions(sessions)